In [1]:
# S5E8_00_Baseline
# 単純モデルの作成(LGBM,XGB,CatBoost)

In [2]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import os

# データフレーム読み込み
train_df = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")

# データ結合
all_df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
max_row = len(all_df)

In [3]:
# all_df.info() # 特徴量、欠損、型確認

In [4]:
# test_df.head()

In [5]:
# 数値列とカテゴリ列を取得
num_col = []
cat_col = []

train_df2 = train_df.drop(["id","y"],axis=1)

for col in train_df2.columns:
    if train_df2[col].dtypes!="object":
        num_col.append(col)
    else:
        cat_col.append(col)

In [6]:
num_col

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

In [7]:
# all_df.head(1)

In [8]:
# 訓練データとテストデータに分離
train = all_df[:len(train_df)]
test = all_df[len(train_df):]

# 訓練データをx,yに分割
X = train.drop(["id","y"],axis=1)
y = train["y"]

In [9]:
# ラベルエンコード
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

# 訓練データでfit
LE.fit(train_df[num_col])

# 訓練・テストデータをそれぞれtransform
LE_train = LE.transform(train_df[col])
LE_test = LE.transform(test_df[col])

# データフレーム変換
LE_train_df = pd.DataFrame(
    LE_train,columns=LE.get_feature_names_out(cat_col))
LE_test_df = pd.DataFrame(
    LE_test,columns=LE.get_feature_names_out(cat_col))

ValueError: y should be a 1d array, got an array of shape (750000, 7) instead.

In [ ]:
###################################################
############ Light GBM ############################
###################################################
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# 学習、バリデーションデータ
pred_lgb = np.zeros(len(train_df))
models_lgb = []

# 評価履歴を格納する辞書
evals_result_lgb = {}

# パラメータ
lgbm_params = {
    'objective': 'binary',
    "device": "cpu",
    # "device": "gpu",
    'metric': 'auc',
    'verbose': -1,              # ログ出力の制御
    'boosting_type': 'gbdt',
    # 'learning_rate': 0.01,
    # 'feature_fraction': 0.6956717916553479,
    # 'num_leaves':       153,
    # 'bagging_fraction': 0.5279852787927486,
    # 'bagging_freq':     4,
    # 'lambda_l1':        0.004603414256652151,
    # 'lambda_l2':        0.013858762846118894,
    # 'min_data_in_leaf': 87,
    }

# クロスバリデーション
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold,(train_idx,valid_idx) in enumerate(kf.split(X)):

    print("#"*25)
    print(f"### Folf {fold+1}")
    print("#"*25)
    
    X_train_kf = X.iloc[train_idx,:]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx,:]
    y_valid_kf = y.iloc[valid_idx]
    
    # データセット作成
    lgb_train_lgb = lgb.Dataset(X_train_kf,y_train_kf)
    lgb_valid_lgb = lgb.Dataset(X_valid_kf,y_valid_kf)


    # 学習
    model_lgb = lgb.train(
        lgbm_params,
        lgb_train_lgb,
        num_boost_round=2000,
        valid_sets=[lgb_train_lgb,lgb_valid_lgb],
        valid_names=["train","valid"],
        callbacks=[
            lgb.early_stopping(stopping_rounds=100,verbose=False),
            lgb.record_evaluation(evals_result_lgb),
            lgb.log_evaluation(100),])

    # 各foldでのバリデーション予測
    pred_lgb[valid_idx] = model_lgb.predict(
        X_valid_kf, num_iteration=model_lgb.best_iteration)
    
    # モデルの追加
    models_lgb.append(model_lgb)

In [ ]:
model_lgb.predict(X, num_iteration=models_lgb[0].best_iteration)


In [ ]:
# 訓練データのスコア
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

AUC_lgb = roc_auc_score(y,pred_lgb)
F1_lgb = f1_score(y,np.round(pred_lgb,0))
print(f"LGB: AUC score = {AUC_lgb}, F1 = {F1_lgb}")

# 学習曲線
lgb.plot_metric(evals_result_lgb,
                title="LightGBM AUC",)

In [ ]:
#################################################
############ XGBoost ############################
#################################################
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# 学習、バリデーションデータ
pred_xgb = np.zeros(len(train_df))
models_xgb = []

# 評価履歴を保存する辞書
evals_result_xgb = {}

# パラメータ
xgb_params = {
    "objective": "binary:logistic",
    "tree_method": "hist",
    # "tree_method": "gpu_hist",
    # "gpu_id": 0,
    "eval_metric": "auc",
}

# クロスバリデーション
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X)):

    print("#"*25)
    print(f"### Fold {fold+1}")
    print("#"*25)

    # foldごとの訓練、バリデーションデータ
    X_train_kf = X.iloc[train_idx,:]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx,:]
    y_valid_kf = y.iloc[valid_idx]

    # DMatrixに変換
    dtrain = xgb.DMatrix(X_train_kf,label=y_train_kf)
    dvalid = xgb.DMatrix(X_valid_kf,label=y_valid_kf)

    # 学習
    model_xgb = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=2000,
        evals=[(dtrain,"train"),(dvalid,"valid")],
        early_stopping_rounds=100,
        evals_result_xgb=evals_result,
        verbose_eval=100,
    )

    # 各foldでのバリデーション予測
    pred_xgb[valid_idx] = model_xgb.predict(
        dvalid,
        iteration_range=(0,model_xgb.best_iteration+1))

    # モデルの追加
    models_xgb.append(model_xgb)

In [ ]:
# 訓練データのスコア
from sklearn.metrics import f1_score

AUC_xgb = roc_auc_score(y,pred_xgb)
F1_xgb = f1_score(y,np.round(pred_xgb,0))
print(f"XGB: AUC score = {AUC_xgb}, F1 = {F1_xgb}")

In [ ]:
# 学習曲線の描画
from matplotlib.pyplot as plt
epochs = len(evals_result_xgb['train']['auc'])
x_axis = range(0, epochs)

plt.figure()
plt.plot(x_axis, evals_result_xgb['train']['auc'], label='Train')
plt.plot(x_axis, evals_result_xgb['valid']['auc'], label='Validation')
plt.xlabel('Iteration')
plt.ylabel('AUC')
plt.title('XGBoost AUC')
plt.grid()
plt.legend()
plt.show()

In [ ]:
#################################################
############ CatBoost ############################
#################################################
from catboost import Pool, train
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# 学習、バリデーションデータ
pred_cb = np.zeros(len(train_df))
models_cb = []

# 履歴を保存
cb_auc_valid = []

cat_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    # "task_type": "GPU",
    # "devices": "0",
    "iterations": 2000,
    "verbose": 100,
}

# クロスバリデーション
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx,valid_idx) in enumerate(kf.split(X)):

    print("#"*25)
    print(f"### Fold {fold+1}")
    print("#"*25)
    
    X_train_kf = X.iloc[train_idx,:]
    y_train_kf = y.iloc[train_idx]
    X_valid_kf = X.iloc[valid_idx,:]
    y_valid_kf = y.iloc[valid_idx]    

    # object型をカテゴリ型に変換
    for col in cat_col:
        X_train_kf.loc[:,col] = X_train_kf.loc[:,col].astype("category")
        X_valid_kf.loc[:,col] = X_valid_kf.loc[:,col].astype("category")

    # データセット設定
    train_pool = Pool(X_train_kf,y_train_kf,cat_features=cat_col)
    valid_pool = Pool(X_valid_kf,y_valid_kf,cat_features=cat_col)

    # 学習
    model_cb = train(
        params=cat_params,
        dtrain=train_pool,
        eval_set=valid_pool,
        early_stopping_rounds=100)

    # 各foldでのバリデーション予測
    pred_cb[valid_idx] = model_cb.predict(X_valid_kf,
        ntree_end=model_cb.best_iteration_,
        prediction_type="Probability"
    )[:,1]
    

    # モデルを追加
    models_cb.append(model_cb)

    evals_result = model_cb.get_evals_result()
    cb_auc_valid.append(evals_result["validation"]["AUC"])

In [ ]:
from sklearn.metrics import f1_score

# スコア表示
AUC_cb = roc_auc_score(y,pred_cb)
F1_cb = f1_score(y,np.round(pred_cb,0))
print(f"CB: AUC score = {AUC_cb}, F1 = {F1_cb}")

# 学習履歴を一番短いfoldに揃える
min_len = min(len(m) for m in cb_auc_valid)
cb_auc_score = [m[:min_len] for m in cb_auc_valid]

# foldごとの結果を平均する
cb_auc_score = np.average(cb_auc_score,axis=0)

import  matplotlib.pyplot as plt
# 履歴の可視化
plt.plot(cb_auc_score, label='Validation')
plt.xlabel('Iteration')
plt.ylabel('AUC')
plt.grid()
plt.legend()
plt.title("CabBoost AUC")
plt.show()

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [ ]:
# pytorch実装
import torch # Tensorの作成や操作
import torch.nn as nn # ニューラルネットワーク
import torch.nn.functional as F # 関数をメソッドとして提供
import torch.optim as optim # オプティマイザ
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# GPUの使用状況確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 設定
SEED = 42
CATEGORICAL = ['job','marital','education','default','housing',
                'loan','contact','month','poutcome']
NUMERICAL = ['age','balance','day','duration','campaign',
             'pdays','previous']
TAEGET = "y"
USE = cat_col + num_col
df_train = train_df.drop("id",axis=1)
df_test = test_df.drop("id",axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# 前処理関数
def preprocessing(df_train, df_test, cat_cols=CATEGORICAL, num_cols=NUMERICAL, target=TARGET):

    # 訓練データ + テストデータ
    df = pd.concat([df_train.drop(columns=target), df_test])
    y = df_train[target]
    train_len = len(df_train)
    
    # 欠損埋め
    df[cat_cols] = df[cat_cols].fillna('None')
    df[num_cols] = df[num_cols].fillna(0)

    # 標準化
    scaler = StandardScaler()
    scaler.fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])
    
    # ラベルエンコーダ
    for col in df.columns:
        if col in cat_cols:
            df[col] = LabelEncoder().fit_transform(df[col])
            df[col]= df[col].astype('category')
            
    return pd.concat([df.iloc[:train_len], y], axis=1), df.iloc[train_len:]

# 前処理の実施
df_train, df_test = preprocessing(df_train, df_test)

In [ ]:
# out-of-fold
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_train.drop(columns=TARGET), df_train[TARGET], test_size=0.20, random_state=SEED, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# embedding
cat_sizes = [len(df_train[col].cat.categories) for col in cat_col]
emb_sizes = [(size, min(50, (size+1)//2)) for size in cat_sizes]
emb_sizes

In [ ]:
from torch.utils.data import Dataset

# データセット関数
class ClassificationColumnarDataset(Dataset):

    # オブジェクト定義
    def __init__(self, df, target, cat_cols=CATEGORICAL):
        self.df_cat = df[cat_cols]
        self.df_num = df.drop(cat_cols, axis=1)
        self.X_cats = self.df_cat.values.astype(np.int64)
        self.X_nums = self.df_num.values.astype(np.float32)
        self.target = target.values.astype(np.int64)

    # データセットのサイズを返す
    def __len__(self):
        return len(self.target)

    # 指定したインデックスのデータとラベルを返す
    def __getitem__(self, idx):
        return [self.X_cats[idx], self.X_nums[idx], self.target[idx]]

In [ ]:
# データセット作成
train_dataset = ClassificationColumnarDataset(X_train, y_train)
val_dataset = ClassificationColumnarDataset(X_val, y_val)
test_dataset = ClassificationColumnarDataset(df_test, pd.Series(np.zeros(len(df_test)).astype(np.int64)))

In [ ]:
from torch.utils.data import DataLoader

# データローダー作成
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
# NNモデル作成
class TabularModel(nn.Module):

    # ネットワーク構造の定義
    def __init__(self, embedding_sizes, n_num):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories, size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings)
        self.n_emb, self.n_num = n_emb, n_num
        self.lin1 = nn.Linear(self.n_emb + self.n_num, 100)
        self.lin2 = nn.Linear(100, 70)
        self.lin3 = nn.Linear(70, 2)
        self.bn1 = nn.BatchNorm1d(self.n_num)
        self.bn2 = nn.BatchNorm1d(100)
        self.bn3 = nn.BatchNorm1d(70)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
 
    # 順伝播
    def forward(self,x_cat,x_num):
        x = [e(x_cat[:, i]) for i, e in enumerate(self.embeddings)]
        x = torch.cat(x, dim=1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_num)
        x = torch.cat([x, x2], dim=1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [ ]:
# インスタンス化
model = TabularModel(emb_szs, len(num_col)).to(device)

# 最適化手法の選択
optimizer = torch.optim.Adam(model.parameters(),
                            lr=0.001,
                            betas=(0.9, 0.999),
                            amsgrad=True)

# 目的関数
compute_loss = nn.CrossEntropyLoss()

In [ ]:
from tqdm.notebook import tqdm
hist = {'train_loss': [], 'train_accuracy': [], 'val_loss': [], 'val_accuracy': []}
EPOCHS = 10
for epoch in range(EPOCHS):
    # 学習
    train_loss = 0.
    train_acc = 0.
    all_labels_train = []
    all_preds_train = []

    model.train()
    train_progress = tqdm(train_dataloader, total=len(train_dataloader), leave=False)
    for batch_idx, (cat_data, num_data, target) in enumerate(train_progress):
        train_progress.set_description(f'<Train> Epoch{epoch+1}')
        cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(cat_data, num_data)
        loss = compute_loss(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy_score(target.tolist(), output.argmax(dim=-1).tolist())
        all_labels_train += target.tolist()
        all_preds_train += output.tolist()

        train_progress.set_postfix(loss=train_loss/(batch_idx+1), acc=train_acc/(batch_idx+1))

    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataloader)

    # 検証
    val_loss = 0.
    val_acc = 0.
    best_acc = 0.
    all_labels_val = []
    all_preds_val = []    

    model.eval()
    val_progress = tqdm(val_dataloader, total=len(val_dataloader), leave=False)
    with torch.no_grad():
        for batch_idx, (cat_data, num_data, target) in enumerate(val_progress):
            val_progress.set_description(f'<Val> Epoch{epoch+1}')
            cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
            output = model(cat_data, num_data)
            loss = compute_loss(output, target)

            val_loss += loss.item()
            val_acc += accuracy_score(target.tolist(), output.argmax(dim=-1).tolist())
            all_labels_val += target.tolist()
            all_preds_val += output.tolist()

            val_progress.set_postfix(loss=val_loss/(batch_idx+1), acc=val_acc/(batch_idx+1))
        
        val_loss /= len(val_dataloader)
        val_acc /= len(val_dataloader)

    # modelの保存
    # if val_acc > best_acc:
    #     best_acc = val_acc
    #     torch.save(model.state_dict(), MODELS_DIR + f'best_NN_0.pth')


    # 学習状況を保存
    hist['train_loss'].append(train_loss)
    hist['train_accuracy'].append(train_acc)
    hist['val_loss'].append(val_loss)
    hist['val_accuracy'].append(val_acc)

In [ ]:
# 検証データの誤差の可視化
train_loss = hist['train_loss']
val_loss = hist['val_loss']

fig = plt.figure()
plt.rc('font', family='serif')
plt.plot(range(len(train_loss)), train_loss, linewidth=1, label='train_loss')
plt.plot(range(len(val_loss)), val_loss, color='red', linewidth=1, label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(f'validation data best acc : {best_acc}')

# 検証データの精度の可視化
train_acc = hist['train_accuracy']
val_acc = hist['val_accuracy']

fig = plt.figure()
plt.rc('font', family='serif')
plt.plot(range(len(train_acc)), train_acc, linewidth=1, label='train_acc')
plt.plot(range(len(val_acc)), val_acc, color='red', linewidth=1, label='val_acc')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # 数値列の標準化
# scaler = StandardScaler()
# std_train = scaler.fit_transform(X_train[num_col])
# std_valid = scaler.fit_transform(X_valid[num_col])
# # std_test = scaler.transform(test_df[num_col])

# # データフレームに変換
# std_train_df = pd.DataFrame(std_train,columns=num_col)
# std_valid_df = pd.DataFrame(std_valid,columns=num_col)
# # std_test_df = pd.DataFrame(scaled_test,columns=num_col)

In [ ]:
# # One-Hotエンコード(カテゴリデータ)
# from sklearn.preprocessing import OneHotEncoder

# OHE = OneHotEncoder(sparse_output=False,dtype=int,handle_unknown="ignore")

# # 訓練データでfit
# OHE.fit(X_train[cat_col])

# # 訓練・テストに対してtransform
# OHE_train = OHE.transform(X_train[cat_col])
# OHE_valid = OHE.transform(X_valid[cat_col])

# # カラム取得
# OHE_col = OHE.get_feature_names_out(cat_col)

# # データフレーム変換
# OHE_train_df = pd.DataFrame(
#     OHE_train, columns=OHE.get_feature_names_out(cat_col))
# OHE_valid_df = pd.DataFrame(
#     OHE_valid, columns=OHE.get_feature_names_out(cat_col))

In [ ]:
# 訓練データの結合
# com_train_df = pd.concat([std_train_df,OHE_train_df],axis=1)
# y_train_df = pd.DataFrame(y_train,columns=["y"])

# com_valid_df = pd.concat([std_valid_df,OHE_valid_df],axis=1)
# y_valid_df = pd.DataFrame(y_valid,columns=["y"])
# com_train_df = pd.concat([std_train_df,OHE_train_df,
#                           pd.DataFrame(y_train,columns=["y"])],axis=1)
# com_valid_df = pd.concat([std_valid_df,OHE_valid_df,
#                           pd.DataFrame(y_valid,columns=["y"])],axis=1)

In [ ]:
# std_train_tensor = torch.tensor(std_train_df.values,
#                                   dtype=torch.float32)
# std_valid_tensor = torch.tensor(std_valid_df.values,
#                                   dtype=torch.float32)
# OHE_train_tensor = torch.tensor(OHE_train_df.values,
#                                   dtype=torch.long)
# OHE_valid_tensor = torch.tensor(OHE_valid_df.values,
#                                   dtype=torch.long)
# lb_train_tensor = torch.tensor(y_train.values,
#                                   dtype=torch.float32)
# lb_valid_tensor = torch.tensor(y_valid.values,
#                                   dtype=torch.float32)

In [ ]:
# Embeddingは次回にする
# from sklearn.preprocessing import LabelEncoder
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# # ラベルエンコード
# for col in cat_col:
#     le = LabelEncoder()
#     all_df[col] = le.fit_transform(all_df[col].values)

#     # Tensor化
#     all_df[col] = torch.tensor(all_df[col],dtype=torch.long)

#     # Embedding層に入力
#     num_class = len(le.classes_)
#     embedding_dim = 3 # 埋め込みベクトルの次元数

In [ ]:
# Datasetクラスの定義
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self,num_tensor,cat_tensor,label_tensor):
        self.num = num_tensor
        self.cat = cat_tensor
        self.label = label_tensor

    # データセットのサイズを返す
    def __len__(self):
        return len(self.label)

    # 指定したインデックスのデータとラベルを返す
    def __getitem__(self, idx):
        return self.num[idx],self.cat[idx],self.label[idx]

In [ ]:
# DataLoader作成
from torch.utils.data import DataLoader

# バッチサイズ
bs = 8

feature_cols = com_train_df.columns
target_col = "y"

# データセット
train_dataset = CustomDataset(
    std_train_tensor,OHE_train_tensor,lb_train_tensor)
                              
valid_dataset = CustomDataset(
    std_valid_tensor,OHE_valid_tensor,lb_valid_tensor)

# データローダー
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=True)

In [ ]:
# NNモデル作成
class NeuralNet(nn.Module):

    # 使用するオブジェクトを定義
    def __init__(self,
                 num_num_features,
                 num_categories,
                 embedding_dim=4):
        super(NeuralNet, self).__init__()
        self.num_layer = nn.Linear(num_num_features,8) # 数値特徴処理
        self.emb_layer = nn.Embedding(num_categories,embedding_dim) # カテゴリ特徴処理
        self.fc = nn.Linear(8+embedding_dim,1)

    # 順伝播
    def forward(self,num_x,cat_x):
        num_out = torch.relu(self.num_layer(num_x))
        cat_out = self.emb_layer(cat_x).squeeze(1)
        x = torch.cat([num_out,cat_out],dim=1)
        return torch.sigmoid(self.fc(x))

# インスタンス化
model = NeuralNet(num_num_features=7,
                  num_categories=44)

# 最適化手法
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)

# 損失関数
criterion = nn.BCELoss()

In [ ]:
# エポックの数
max_epoch = 5

# モデルの初期化
torch.manual_seed(42)

# 訓練データ結果
# 検証データ結果

# 学習ループ
for epoch in range(max_epoch):

    # 訓練モード
    model.train()
    
    for batch_num,batch_cat,batch_label in train_loader:

        # パラメータの勾配を初期化
        optimizer.zero_grad() 

        # 入力データとラベルをGPU用に変換
        # x = x.to(device)
        # t = t.to(device)
        
        # 順伝播 (batch_size,1 -> (batch_size,))
        preds = model(batch_num,batch_cat).squeeze()

        # ロス計算
        loss = criterion(preds,batch_label)

        # 各パラメータの勾配を算出
        loss.backward()   
        
        # パラメータ更新    
        optimizer.step()   

    # 評価モード
    model.eval()

    # 訓練データの計算
    tmp_xtrain = x_train_tensor.to(device)
    tmp_ytrain = y_train_tensor.to(device)    
    # valデータの計算
    tmp_xvalid = x_valid_tensor.to(device)
    tmp_yvalid = y_valid_tensor.to(device)  

    # 予測
    y1 = model(tmp_xtrain)
    loss1 = criterion(y1,tmp_ytrain)

    y2 = model(tmp_xvalid)
    loss2 = criterion(y2,tmp_yvalid)

    # 最大値を抽出
    y_pred1 = torch.argmax(y1,1)
    y_pred2 = torch.argmax(y2,1)